In [1]:
import itertools
import os

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
def articles():
    with open('page_revisions_text', 'rb') as text_file:
        pending_article_data = b''
        while True:
            data = text_file.read(1024 ** 2)
            if len(data) == 0:
                break

            articles = data.split(b'\0')
            articles[0] = pending_article_data + articles[0]
            for index, article in enumerate(articles):
                if index + 1 == len(articles):
                    pending_article_data = article
                else:
                    yield article

        print(pending_article_data)
        if len(pending_article_data) != 0:
            yield pending_article_data

In [3]:
subword_text_encoder = tfds.features.text.SubwordTextEncoder.load_from_file('vocab_4096')

In [4]:
BATCH_SIZE = 192
BATCHED_ITEM_LENGTH = 256
BUFFER_SIZE = 1024
TYPE=np.int16

def articles_generator():
    for index, article in enumerate(itertools.islice(articles(), 0, 10000)):
        yield np.array(subword_text_encoder.encode(article + b'\0'), dtype=TYPE)

    # Pad the article count to the batch size
    # We do this to ensure that no data is dropped
    index += 1
    while index % BATCH_SIZE != 0:
        yield np.array([0], dtype=TYPE)
        index += 1

def subbatches():
    dataset = tf.data.Dataset.from_generator(articles_generator, output_types=TYPE)
    dataset = dataset.shuffle(BUFFER_SIZE)
    dataset = dataset.padded_batch(BATCH_SIZE, padded_shapes=([None]), drop_remainder=True)
    dataset = dataset.shuffle(2000)

    for batch in dataset.as_numpy_iterator():
        remaining = batch
        while remaining.shape[1] > 1:
            yield remaining[:, :BATCHED_ITEM_LENGTH + 1]
            remaining = remaining[:, BATCHED_ITEM_LENGTH:]

dataset = tf.data.Dataset.from_generator(subbatches, output_types=TYPE, output_shapes=(BATCH_SIZE, None))
dataset = dataset.map(lambda batch: (batch[:, :-1], batch[:, 1:]))

dataset

<MapDataset shapes: ((192, None), (192, None)), types: (tf.int16, tf.int16)>

In [5]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    return tf.keras.Sequential([
        tf.keras.layers.Masking(mask_value=0, batch_input_shape=[batch_size, None]),
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size),
    ])

In [6]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

def average_batch_length(true_labels, predictions):
    return tf.shape(true_labels)[1]

model = build_model(vocab_size = subword_text_encoder.vocab_size, embedding_dim=512, rnn_units=1024, batch_size=BATCH_SIZE)
model.compile(optimizer='adam', loss=loss, metrics=[average_batch_length])

In [7]:
checkpoint_dir = './training_checkpoints-1' # Directory where the checkpoints will be saved
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}") # Name of the checkpoint files

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)

In [8]:
class ModelStateResetter(tf.keras.callbacks.Callback):
    def __init__(self):
        self.last_total_length = 0

    def on_batch_end(self, batch, logs={}):
        average_batch_length = logs.get('average_batch_length', 0)
        total_length = int(round(average_batch_length * (batch + 1)))
        current_batch_length = total_length - self.last_total_length
        self.last_total_length = total_length
        
        if current_batch_length < BATCHED_ITEM_LENGTH:
            self.model.reset_states()
        
model_state_resetter_callback = ModelStateResetter()

total_epochs = 10

for epoch in range(total_epochs):
    print('Epoch %d/%d' % (epoch + 1, total_epochs))
    model.fit(dataset, callbacks=[checkpoint_callback, model_state_resetter_callback])

Epoch 1/10
4999/4999 [==============================] - 4192s 838ms/step - loss: 0.4641 - average_batch_length: 254.7195
Epoch 2/10
5045/5045 [==============================] - 4098s 812ms/step - loss: 0.3454 - average_batch_length: 254.6313
Epoch 3/10
4993/4993 [==============================] - 4031s 807ms/step - loss: 0.3219 - average_batch_length: 254.6477
Epoch 4/10
4986/4986 [==============================] - 4032s 809ms/step - loss: 0.3073 - average_batch_length: 254.7595
Epoch 5/10
5089/5089 [==============================] - 4101s 806ms/step - loss: 0.2915 - average_batch_length: 254.6630
Epoch 6/10
5088/5088 [==============================] - 4108s 807ms/step - loss: 0.2846 - average_batch_length: 254.6800
Epoch 7/10
4969/4969 [==============================] - 4006s 806ms/step - loss: 0.2868 - average_batch_length: 254.7016
Epoch 8/10
4906/4906 [==============================] - 3961s 807ms/step - loss: 0.2848 - average_batch_length: 254.5351
Epoch 9/10
4989/4989 [==========

In [9]:
total_epochs = 10

for epoch in range(total_epochs):
    print('Epoch %d/%d' % (epoch + 1, total_epochs))
    model.fit(dataset, callbacks=[checkpoint_callback, model_state_resetter_callback])

Epoch 1/10
5000/5000 [==============================] - 4079s 816ms/step - loss: 0.2707 - average_batch_length: 254.6328
Epoch 2/10
4955/4955 [==============================] - 4039s 815ms/step - loss: 0.2708 - average_batch_length: 254.7427
Epoch 3/10
5049/5049 [==============================] - 4101s 812ms/step - loss: 0.2636 - average_batch_length: 254.6243
Epoch 4/10
5065/5065 [==============================] - 4151s 820ms/step - loss: 0.2617 - average_batch_length: 254.6235
Epoch 5/10
5122/5122 [==============================] - 4210s 822ms/step - loss: 0.2581 - average_batch_length: 254.7558
Epoch 6/10
4967/4967 [==============================] - 4077s 821ms/step - loss: 0.2708 - average_batch_length: 254.5917
Epoch 7/10
   3535/Unknown - 2784s 787ms/step - loss: nan - average_batch_length: 254.7943

KeyboardInterrupt: 